# Dumping radar data in a CF compliant format

This notebook describes how to create a netCDF4 file with the output of our radar data following the conventions described in http://cfconventions.org/documents.html V1.7.


For future reference, http://wiki.esipfed.org/index.php/NetCDF-CF_File_Examples_for_Satellite_Swath_Data
is an example on how to deal with satellite data.

In [ ]:
from netCDF4 import Dataset
from tdm.radar import utils
from datetime import datetime
import numpy as np
import os
import time

In [ ]:
def setncattr(o, attrs):
    for k, v in attrs.items():
        o.setncattr(k, v)

def showattrs(o):
    for k in o.ncattrs():
        print(k, o.getncattr(k))

In [ ]:
root = './data/radarsample/cag01est2400/'
template = './data/radarsample/radarfootprint.tif'

In [ ]:
ga = utils.GeoAdapter(template)

In [ ]:
xpos, ypos = ga.xpos(), ga.ypos()

In [ ]:
radar_images = utils.get_images(root)
first_tstamp = radar_images[0][0]
n_images = len(radar_images)

In [ ]:
try:
    os.unlink('radar.nc')
except FileNotFoundError:
    pass
r = Dataset('radar.nc', 'w')

In [ ]:
global_attributes = {
    'Conventions': 'CF-1.7',
    'title': 'Rainfall Radar acquisitions',
    'institution': 'AEN inc',
    'source': 'Radar XXX',
    'references': 'http://www.tdm-project.it/en/XXX/radar',
    'history': 'Conversion from raw data using XXXX'
}
setncattr(r, global_attributes)

In [ ]:
showattrs(r)

In [ ]:
# See 4.4 time coordinate
timed = r.createDimension('time', n_images)
xd = r.createDimension('x', ga.cols)
yd = r.createDimension('y', ga.rows)

# See section 5.1 and https://code.mpimet.mpg.de/boards/1/topics/5765

times = r.createVariable('time', 'f4', (timed.name,))
setncattr(times, {'long_name': 'time',
                  'units': 'minutes since %s UTC' % first_tstamp})

x = r.createVariable('x', 'f4', (xd.name,))
setncattr(x, {'long_name': 'x coordinate of projection',
              'standard_name': 'projection_x_coordinate',
              'units': 'km'})

y = r.createVariable('y', 'f4', (yd.name,))
setncattr(y, {'long_name': 'y coordinate of projection',
              'standard_name': 'projection_y_coordinate',
              'units': 'km'})

lat = r.createVariable('lat', 'f4', (xd.name, yd.name))
setncattr(lat, {'long_name': 'latitude coordinate',
                'standard_name': 'latitude',
                'units': 'degrees_north'})

lon = r.createVariable('lon', 'f4', (xd.name, yd.name))
setncattr(lon, {'long_name': 'longitude coordinate',
                'standard_name': 'longitude',
                'units': 'degrees_east'})

In [ ]:
# See section 5.6.1
# FIXME this should be, in principle, derived from the wkt
crs = r.createVariable('crs', 'i4') # a dummy scalar used as anchor for the crs
setncattr(crs, {
 'grid_mapping_name': 'transverse_mercator',
 'longitude_of_central_meridian': 9.0,
 'latitude_of_projection_origin': 0.0,
 'false_easting': 1500000.0,
 'false_northing': 0.0,
 'scale_factor_at_central_meridian': 0.9996,
 'semi_major_axis': 6378388.0,
 'inverse_flattening': 297,
 'projected_coordinate_system_name': 'EPSG:3003 Monte Mario / Italy zone 1',
 'geographic_coordinate_system_name': 'Monte Mario',
 'horizontal_datum_name':  'Monte_Mario',
 'reference_ellipsoid_name': 'International 1924',
 'prime_meridian_name': "Greenwich",
 'towgs84': [-104.1,-49.1,-9.9,0.971,-2.917,0.714,-11.68],
 'crs_wkt': ga.wkt
})

In [ ]:
rf_rate = r.createVariable('rainfall_rate', 'f4', 
                           (timed.name, xd.name, yd.name), zlib=True, 
                           chunksizes=(60, ga.cols, ga.rows),
                           fill_value=utils.RAINFALL_FILL_VALUE)

In [ ]:
setncattr(rf_rate, {'long_name': 'estimated rainfall rate',
                    'standard_name': 'rainfall_rate',
                    'coordinates': 'lat lon',
                    'grid_mapping': 'crs',
                    'units': 'mm/hour'})

In [ ]:
rf_rate.get_var_chunk_cache()

```
set_var_chunk_cache(self,size=None,nelems=None,preemption=None)

ncid
    NetCDF ID, from a previous call to nc_open or nc_create.
varid
    Variable ID.
size
    The total size of the raw data chunk cache, in bytes. This should be big enough to hold multiple chunks of data.
nelems
    The number of chunk slots in the raw data chunk cache hash table. This should be a prime number larger than the number of chunks that will be in the cache.
preemption
    The preemtion value must be between 0 and 1 inclusive and indicates how much chunks that have been fully read are favored for preemption. A value of zero means fully read chunks are treated no differently than other chunks (the preemption is strictly LRU) while a value of one means fully read chunks are always preempted before other chunks.
```    
**So the chunk cache is too small**

In [ ]:
rf_rate.set_var_chunk_cache(5 * 60 * 1024 * 1024 * 4, 5, 1.0)

In [ ]:
x[:] = xpos
y[:] = ypos
lat[:], lon[:] = utils.get_lat_lon(ga.sr, xpos, ypos)

In [ ]:
start = time.time()
times[:] = [(_[0] - first_tstamp).total_seconds() / 60 for _ in radar_images]
for i, (ts, fname) in enumerate(radar_images):
    signal = utils.get_image_data(fname)
    rf_rate[i, :, :] = utils.estimate_rainfall(signal)
r.close()
print(time.time() - start)

on my laptop, no compression 8.74, with compression 11.90

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
r = Dataset('radar.nc', 'r')

In [ ]:
print(sorted(r.variables.keys()))

In [ ]:
r.variables["time"]

In [ ]:
r.variables["time"][:5].data

In [ ]:
r.variables["rainfall_rate"]

In [ ]:
rainfall = r.variables["rainfall_rate"][0].data
x, y = r.variables["x"][:].data, r.variables["y"][:].data

In [ ]:
plt.figure(dpi=144)
c = plt.contourf(rainfall, levels=np.arange(0, 3, 0.1))
cbar = plt.colorbar(c)
cbar.ax.set_ylabel('rainfall (mm/h)')
ax = plt.gca()
ax.set_aspect("equal")
ax.set_xticks([0, rainfall.shape[0]])
ax.set_xticklabels(["%d" % x[0], "%d" % x[-1]])
ax.set_yticks([0, rainfall.shape[1]])
ax.set_yticklabels(["%d" % y[-1], "%d" % y[0]])

In [ ]:
r.close()